# Homework 8
In this homework, you will explore neural networks, writing code to complete forward and backward passes for a fully-connected network. Having done this, you will train these networks with the goal of classifying images of apparel into the type (from the Fashion MNIST dataset). You will finally upload your predictions on the test set we give you to [this kaggle competition](https://www.kaggle.com/t/e7ec8dd1f8e64aa2997d82edd5e8f6eb) (more details later in the homework).

In the (optional) extension section at the end, you will see how to use [PyTorch](https://pytorch.org/) to implement, train, and evaluate a convolutional neural network.

There are a number of programming **tasks** and **quiz questions** in this homework.
- For **tasks**, you will need to either **add code between comments "`#### TASK N CODE`"** to complete them or **modify code between those comments**. **DO NOT delete the comments "#### TASK N CODE". This is for graders' reference and you might not get full points if you tamper with these comments.**
- For **quiz questions**, you will need to answer in a few sentences between the given lines.
- For **optional tasks**, you are **NOT required to turn them in**. However, we encourage you to complete them as they are good practice.
- For **challenge-optional tasks**, you are **NOT required to turn them in**. However, you will receive extra credit for completing the challenge.

-----

We will first import libraries.
- We provide you with a few relevant functions in the `utils` module. The `linclass` module has a bunch of linear classifiers that we also provided you in the previous homework.
- In this assignment, we will program neural networks with different network operations represented as classes. Thus, we will need a common "abstract" class and several abstract methods; `abc` module provides these.

In [1]:
import abc
from typing import List, Optional, Tuple

import linclass
import numpy as np
import matplotlib.pyplot as plt
import utils

In [2]:
SEED = 0
np.random.seed(0)

## Neural Networks

As discussed in lecture, neural networks are Directed Acyclic Graphs (DAGs) with input and output nodes, where each node is associated with an "activation" function, and where the edges have weights. Neural networks can be represented as DAGs in code using a "computational graph". In fact, many libraries such as PyTorch and Tensorflow choose to do so.

We will follow a code structure similar to PyTorch, though we will limit the scope of this homework to a path in a DAG. A path in a DAG corresponds to a Sequential neural network, called as such because it comprises of a sequence of layers (collection of units of nodes) where the output of one layer is the input of the next layer in the sequence.

We start by defining a `Module` that defines a node or a collection of nodes. A `Module` is an abstract view of the fundamental element of a neural network. It defines several attributes and methods, some of which must be implemented by the derived classes.
- `train` is a flag to denote whether the module is in training or evaluation mode. We will see how this is important later. This flag is controlled by `train_mode()` and `eval_mode()` methods.
- `forward()` and `backward()` methods do forward and backward passes of the module (recall the discussion from recitation).
    - Inside `forward()`, attributes `_input` and `_output` must be assigned. These correspond to the input of the module (`x`) and the output (after whatever operations the module does in the forward pass).
    - Inside `backward()`, attributes `_grad_output` and `grad_input` must be assigned. These correspond to the gradient of the loss w.r.t. module's outputs and module's inputs respectively. To do the backward pass, attributes `_input` and `_output` containing the values of the forward pass might come handy.
- `get_params()` and `set_params()` are getter and setter functions for _all the **trainable** parameters_ of the module.
- `get_grad_params()` method returns the gradient of the loss w.r.t. the **trainable** parameters.


In [3]:
class Module(abc.ABC):
    '''
    A module defines a sub-graph G' = (V, E) in the DAG G that represents a
    neural network. Therefore G' is a subset of G. An instantiation of a module
    thus represents a realization of a subgraph, comprising of a set of nodes and
    their connections.
    '''
    def __init__(self):
        # Train mode or eval mode
        self.train = True

        # Forward pass cache
        self._input = None
        self._output = None

        # Backward pass cache
        self._grad_output = None
        self._grad_input = None

    @abc.abstractmethod
    def forward(self, x: np.ndarray):
        '''
        Computes the forward pass z = f(x) where f is the function represented by
        the module, x is the input, and z is the output of the forward pass.

        Assigns to attributes self._input and self._output.

        Args:
            x: Data features. shape (m, in_dim) where m is the number of data
                points and in_dim is the number of input features.
        '''
        pass

    def _check_forward_attrs(self):
        '''Sanity check after a forward pass.'''
        assert self._input is not None
        assert self._output is not None

    @abc.abstractmethod
    def backward(self, grad_output: np.ndarray):
        '''
        Computes the gradient of loss w.r.t. cached input and trainable parameters.

        Assigns to attributes self._grad_output and self._grad_input.
        The gradients w.r.t. trainable parameters must also be cached so that
        they can be returned by self.get_grad_params().

        Args:
            grad_output: Gradient of loss w.r.t. output z of the module, dL/dz.
                shape (m, out_dim) where m is the number of data points and
                out_dim is the number of output features.
        '''
        pass

    def _check_backward_attrs(self):
        '''Sanity check after a backward pass.'''
        assert self._grad_output is not None
        assert self._grad_input is not None

    def get_params(self) -> Optional[np.ndarray]:
        '''
        Returns the trainable parameters of the module. If there are no trainable
        parameters, returns None.

        Returns:
            arr: (jagged) array of trainable parameters, where the entries are
                differently-sized numpy arrays themselves.
        '''
        return None

    def set_params(self, params: np.ndarray):
        '''
        Sets the trainable parameters to params. If there are no trainable parameters
        to set, raises a RuntimeError.

        Args:
            params: (jagged) array of trainable parameters, in the same order
                as obtained from self.get_params(). The identity operation is satisfied:
                ```
                x = self.get_params()
                self.set_params(x)
                x == self.get_params()
                ```
        '''
        raise RuntimeError('No trainable parameters to set!')

    def get_grad_params(self) -> Optional[np.ndarray]:
        '''
        Returns the gradients of the loss w.r.t. trainable parameters of the module.
        If there are no trainable parameters, returns None.

        Returns:
            arr: (jagged) array of gradients of trainable parameters,
                where the entries are differently-sized numpy arrays themselves.
        '''
        return None

    def train_mode(self):
        '''
        Switches on the training mode. Useful e.g. in Dropout, where the nodes must be
        dropped only during training, not evaluation.
        '''
        self.train = True

    def eval_mode(self):
        '''
        Switches on the evaluation mode. Useful e.g. in Dropout, where the nodes must be
        dropped only during training, not evaluation.
        '''
        self.train = False

### [Task 1] Complete Forward-Backward Prop for a Sequential Network

Next, we will define a `Sequential` architecture, called as such because it allows for implementing a sequence of modules. Each module is called a layer, so that the preceding layer connects to the next layer. Sequential architectures are also called "feed-forward" networks.

In this task, you will finish the `backward()` method. Remember to cache `_grad_output` attribute in the method. Also, store the results in `_grad_input`.

We give you the forward pass as reference: iterate through the layers, doing a forward pass on the output of the preceding layer (accessed using `_output`) to get the output of each layer. The output of the sequential network is the final output.

Similarly, in the backward pass, you should iterate through the layers in a reverse manner, doing a backward pass on the gradients w.r.t. inputs of the succeeding layer (accessed using `_grad_input`) to get the gradients w.r.t. inputs of each layer. The gradients w.r.t. inputs of the sequential network should be the final gradients you calculate in this reverse iteration.

In [4]:
class Sequential(Module):
    '''
    A sequence of modules, representing a DAG path.
    '''
    def __init__(self, layers: List[Module]):
        '''
        Args:
            layers: A list of modules to initialize the sequential network.
        '''
        super(Sequential, self).__init__()
        self.layers = layers

    def add(self, layer: Module):
        '''
        Adds the layer at the end of the sequential network.
        '''
        self.layers.append(layer)

    def forward(self, x: np.ndarray):
        '''
        Computes a forward pass sequentially on the network layers.
        '''
        self._input = x
        n_layers = len(self.layers)
        for i in range(n_layers):
            _output_prev = self._input if i == 0 else self.layers[i-1]._output
            self.layers[i].forward(_output_prev)
        self._output = self._input if n_layers == 0 else self.layers[-1]._output

        self._check_forward_attrs()

    def backward(self, grad_output: np.ndarray):
        '''
        Backpropagates the gradient w.r.t. output of the sequential network,
        computing gradients w.r.t. input and trainable parameters of the network.
        '''
        #### TASK 1 CODE
        self._grad_output = grad_output
        n_layers = len(self.layers)
        go = grad_output
        for i in range(n_layers - 1, -1, -1):
            self.layers[i].backward(go)
            go = self.layers[i]._grad_input
        self._grad_input = go
        #### TASK 1 CODE
        self._check_backward_attrs()

    def get_params(self) -> Optional[np.ndarray]:
        params = []
        for layer in self.layers:
            p = layer.get_params()
            if p is not None:
                params.append(p)

        # Wrap parameters in an array. Just np.array(params) won't work due to broadcasting
        # conflicts: https://stackoverflow.com/a/49119983. So initialize array and then fill.
        arr = np.empty(len(params), dtype=np.ndarray)
        arr[:] = params
        return arr

    def set_params(self, params: np.ndarray):
        # Since params has trainable parameters listed in the same order as
        # get_params() would have returned, follow the same iteration, and call
        # layer.set_params() on params[i] where i is the ith layer with any trainable
        # parameters
        i = 0
        for layer in self.layers:
            p = layer.get_params()
            if p is not None:
                layer.set_params(params[i])
                i += 1

    def get_grad_params(self) -> Optional[np.ndarray]:
        grad_params = []
        for layer in self.layers:
            g = layer.get_grad_params()
            if g is not None:
                grad_params.append(g)
        arr = np.empty(len(grad_params), dtype=np.ndarray)
        arr[:] = grad_params
        return arr

    def train_mode(self):
        # Switch on training in all layers
        super().train_mode()
        for layer in self.layers:
            layer.train_mode()

    def eval_mode(self):
        # Switch on eval in all layers
        super().eval_mode()
        for layer in self.layers:
            layer.eval_mode()

### [Task 2] Complete Forward-Backward Prop for a Linear Layer

A linear layer defines an affine transformation on inputs $x \in \mathbb{R}^{m \times d_{in}}$, that is $z = x W + b$ where $W \in \mathbb{R}^{d_{in} \times d_{out}}, b \in \mathbb{R}^{d_{out}}$ are trainable parameters. This affine transformation defines a "fully-connected module" where each output feature $z[:, j]$ is connected to input feature $x[:, i]$ with an edge-weight $W[i, j]$.

We provide you with code that initializes the weight and bias parameters, and their corresponding attributes for storing gradients.

You will complete the `backward()` method. We provide you with the forward pass as reference.

The discussion from the recitation might be helpful here.

**Note on Initialization**: We initialize i.i.d. $W[i, j] \sim \mathcal{N}\left( 0, \sqrt{\frac{2}{d_{in}}} \right)$ and $b_j = 0$ for all $i, j$. The scaled variance for the weights is important for faster convergence, but this is outside the scope of the course.

In [5]:
class Linear(Module):
    '''
    Linear transformation on the inputs, z = xW + b.

    Corresponds to all nodes in the preceding layer connected to all nodes in the
    current layer.
    '''
    def __init__(self, in_dim: int, out_dim: int):
        '''
        Args:
            in_dim: Number of input dimensions (number of incoming connections
                in the network).
            out_dim: Number of output dimensions (number of outgoing connections
                in the network).
        '''
        super(Linear, self).__init__()

        # Initialize trainable parameters
        self.weight = np.random.normal(0, np.sqrt(2/in_dim), (in_dim, out_dim))
        self.bias =  np.zeros(out_dim)

        # Initialize gradients w.r.t. trainable parameters
        self._grad_weight = None
        self._grad_bias = None

    def forward(self, x: np.ndarray):
        '''
        Args:
            x: Data features. shape (m, in_dim)
        '''
        assert x.shape[1] == self.weight.shape[0]
        
        self._input = x
        self._output = x @ self.weight + self.bias
        self._check_forward_attrs()

    def backward(self, grad_output: np.ndarray) -> np.ndarray:
        '''
        Computes gradient w.r.t. trainable parameters and input and returns the gradient
        w.r.t. input.

        Important: gradients are accumulated for trainable parameters, i.e. added to the existing
        values.

        Args:
            grad_output: Gradient w.r.t. output dL/dz. shape (m, out_dim)

        Returns:
            grad_input: shape (m, in_dim)
        '''
        assert grad_output.shape[1] == self.weight.shape[1]

        #### TASK 2 CODE
        self._grad_output = grad_output
        self._grad_weight = self._input.T @ self._grad_output
        self._grad_bias = grad_output.sum(axis = 0)
        self._grad_input = self._grad_output @ self.weight.T
        #### TASK 2 CODE
        self._check_backward_attrs()

    def _check_backward_attrs(self):
        super()._check_backward_attrs()
        assert self._grad_weight is not None
        assert self._grad_bias is not None

    def get_params(self) -> Optional[np.ndarray]:
        params = np.empty(2, dtype=np.ndarray)
        params[0] = self.weight
        params[1] = self.bias
        return params

    def set_params(self, params: np.ndarray):
        assert len(params) == 2
        self.weight = params[0]
        self.bias = params[1]

    def get_grad_params(self) -> Optional[np.ndarray]:
        grad_params = np.empty(2, dtype=np.ndarray)
        grad_params[0] = self._grad_weight
        grad_params[1] = self._grad_bias
        return grad_params

### [Task 3] Complete Forward-Backward Prop for ReLU Activation

The `Module` interface allows us to use every element of in the network as a node in the graph, even the activation function applied to a node. The `ReLU` activation function, $z = \max(0, x)$ for inputs $x$, does not have any trainable parameters, however.

You will complete the `backward()` method below.

In [6]:
class ReLU(Module):
    '''
    ReLU activation, not trainable. z = max(x, 0) for each input value x.
    '''
    def __init__(self):
        super(ReLU, self).__init__()

    def forward(self, x: np.ndarray):
        '''
        Args:
            x: Data features. shape (m, in_dim)
        '''
        self._input = x
        self._output = np.maximum(0., self._input)
        self._check_forward_attrs()

    def backward(self, grad_output: np.ndarray) -> np.ndarray:
        '''
        Since there are no trainable parameters, only the gradient w.r.t. input is computed.

        Args:
            grad_output: Gradient w.r.t. output dL/dz. Any shape
        '''
        assert grad_output.shape == self._input.shape

        #### TASK 3 CODE
        self._grad_output = grad_output
        self._grad_input = self._grad_output * (self._input > 0)
        #### TASK 3 CODE
        self._check_backward_attrs()

### Regularization using Dropout

We define the final module we will use in this homework: Dropout. Dropout is a commonly used technique for regularization to try to "limit dependence" of the network's prediction capabilities on any specific trainable parameters.

Dropout accomplishes by switching off nodes, i.e. "dropping" them, with some probability during training. Nodes are only dropped during training and not evaluation as Dropout is a regularization technique. Using Dropout during evaluation would cause us to not use the full trained network. This is where the `train` attribute defined in the `Module` abstract class comes in handy. Before training the model, we will call `model.train_mode()` to set `model.train = True`. The implementation `train_mode()` in `Sequential` ensures that the `train` attribute of each layer is set to `True`. Before using the model for evaluation, we will call `model.eval_mode()` to set `model.train = False` and all `train` attributes of the comprising layers to `False`. This way Dropout is not active during evaluation.

The non-dropped values are rescaled so that the expectation of the non-dropped node is the same as when dropout wasn't applied.

Read through the code so you understand how Dropout is implemented.

In [7]:
class Dropout(Module):
    '''
    A dropout layer.
    '''
    def __init__(self, p: float = 0.5):
        '''
        Args:
            p: (default 0.5) Probability of dropping each node (prob. of setting each value to 0).
                If p is 0, then no nodes are dropped, i.e. we get the identity layer.
        '''
        assert 0 <= p <= 1
        super(Dropout, self).__init__()
        self.p = p

    def forward(self, x: np.ndarray):
        self._input = x
        self._output = self._input
        if self.train and (not np.isclose(self.p, 0)):
            # In training mode and drop probability is positive

            # Create a mask to apply to the input using Bernoulli(1-p) RV
            self.mask = np.random.binomial(1, 1 - self.p, x.shape).astype(float)

            # Scale the mask so that the expected value of prediction during
            # testing is same as x, and not (1-p)x
            self.mask /= 1 - self.p

            self._output *= self.mask
        self._check_forward_attrs()

    def backward(self, grad_output: np.ndarray):
        self._grad_output = grad_output
        self._grad_input = self._grad_output
        if self.train and (not np.isclose(self.p, 0)):
            # In training mode and drop probability is positive, _grad_input is masked
            self._grad_input *= self.mask
        self._check_backward_attrs()

### [Task 4] Complete MultiLogisticLoss

A loss function is applied on the predicted probabilities of each label $r \in \mathbb{R}^{m \times k}$ and the true labels $y \in \{0, \dots, k\}^{m}$. Following the code pattern of `Module`, we define the abstract class `Loss`.
- The `forward()` method computes the loss value using responses $r$ and true labels $y$. It assigns to attributes `_input = r`, `_input_target = y`, and `_output`.
- The `backward()` method computes the gradient of the loss w.r.t. the inputs, i.e. the responses `r`. It assigns to attribute `_grad_input`.

In this task, you will complete `MultiLogisticLoss`. The implementation is almost the same as you did in the previous homework.

We provide you an implementation of softmax in `utils.softmax`---you might find this useful.

In [8]:
class Loss(abc.ABC):
    '''Defines a loss function.'''
    def __init__(self, k: int):
        '''
        Args:
            k: Number of labels.
        '''
        self.k = k
        self._input = None
        self._input_target = None
        self._output = None
        self._grad_input = None

    @abc.abstractmethod
    def forward(self, r: np.ndarray, y: np.ndarray):
        '''
        Computes the loss value using responses r and true labels y.

        Sets the attributes self._input, self._input_target, and self._output.

        Args:
            r: Responses of a classifier. shape (m, k) where m is the number of data
                points.
            y: True labels. shape (m). For all i, 0 <= y_i < k
        '''
        pass

    def _check_forward_attrs(self):
        assert self._input is not None
        assert self._input_target is not None
        assert self._output is not None

    @abc.abstractmethod
    def backward(self):
        '''
        Computes the gradient of the loss value w.r.t. cached responses.

        Sets the attribute self._grad_input.
        '''
        pass

    def _check_backward_attrs(self):
        assert self._grad_input is not None


class MultiLogisticLoss(Loss):
    def __init__(self, k: int):
        super(MultiLogisticLoss, self).__init__(k)

    def forward(self, r: np.ndarray, y: np.ndarray):
        '''
        Computes the multiclass logistic loss, using the softmax operation to
        convert responses r to normalized probabilities.
        '''
        assert r.shape[0] == y.shape[0]
        assert r.shape[1] == self.k

        self._input = r
        self._input_target = y

        stable_r = self._input - np.max(self._input, axis=1)[:, np.newaxis]
        nll = np.log(np.sum(np.exp(stable_r), axis=1)) - \
            np.take_along_axis(stable_r, self._input_target[:, np.newaxis], axis=1).flatten()
        self._output = np.mean(nll)
        self._check_forward_attrs()

    def backward(self):
        #### TASK 4 CODE
        self._grad_input = utils.softmax(self._input)
        self._grad_input -= np.eye(self.k)[self._input_target]
        self._grad_input /= len(self._input)
        #### TASK 4 CODE
        self._check_backward_attrs()


### Testing your gradient calculation

To help you ensure your code calculates the forward and backward passes correctly, we have provided a few tests below. These tests match your implementation against a numerical approximation of the gradient. You should get relative errors on the order of $10^{-8}$ or less.

In [9]:
def test_log_grad():
    '''
    Tests MultiLogisticLoss Gradient implementation. The relative error should be small.
    '''
    print('Multi Logistic Loss Grad Test')
    loss = MultiLogisticLoss(k=10)
    x = np.random.rand(3, 10)
    y = np.array([4, 5, 6])
    def test_f(x):
        loss.forward(x, y)
        return loss._output

    # Backprop manually
    test_f(x)
    loss.backward()
    grad_input = loss._grad_input

    # Gradient approximation
    grad_input_num = utils.numeric_grad(test_f, x, 1, 1e-6)

    rel_err = utils.relative_error(grad_input, grad_input_num, 1e-8)
    print(f'Relative err between your computation and numerical gradient: {rel_err:e}')

test_log_grad()

Multi Logistic Loss Grad Test
Relative err between your computation and numerical gradient: 9.176152e-09


In [10]:
class LossForUnitTesting(Loss):
    def __init__(self, k):
        super(LossForUnitTesting, self).__init__(k)

    def forward(self, x: np.ndarray, y: np.ndarray):
        self._input = x
        self._input_target = y
        self._output = np.mean(np.sum(np.abs(self._input), axis=1))
        self._check_forward_attrs()

    def backward(self):
        self._grad_input = np.sign(self._input) / self._input.shape[0]
        self._check_backward_attrs()


def test_module(model: Module):
    '''
    Tests Module Gradient implementation. The relative error should be small.
    '''
    print('Module Grad Test')

    model.eval_mode()

    loss = LossForUnitTesting(k=10)
    x = np.random.rand(4, 10)
    y = np.array([4, 5, 6, 8])
    def test_f(x):
        model.forward(x)
        loss.forward(model._output, y)
        return loss._output

    # Backprop manually
    test_f(x)
    loss.backward()
    model.backward(loss._grad_input)
    grad_input = model._grad_input

    # Gradient approximation
    grad_input_num = utils.numeric_grad(test_f, x, 1, 1e-6)

    rel_err = utils.relative_error(grad_input, grad_input_num, 1e-8)
    print(f'Relative err between your computation and numerical gradient: {rel_err:e}')

print('Testing Linear...')
model = Linear(10, 20)
test_module(model)

print('Testing ReLU...')
model = ReLU()
test_module(model)

print('Testing Dropout...')
model = Dropout()
test_module(model)

print('Testing 2-layer model')
model = Sequential([
    Linear(10, 20),
    ReLU(),
    Linear(20, 10)
])
test_module(model)

Testing Linear...
Module Grad Test
Relative err between your computation and numerical gradient: 3.063965e-08
Testing ReLU...
Module Grad Test
Relative err between your computation and numerical gradient: 5.139782e-10
Testing Dropout...
Module Grad Test
Relative err between your computation and numerical gradient: 5.139782e-10
Testing 2-layer model
Module Grad Test
Relative err between your computation and numerical gradient: 3.919368e-09


### [Task 5] Building a Neural Network Classifier

We can now define a classifier using the code structure we have followed in the previous homeworks. A neural network classifier consists of an initialized model and a loss, specified when instantiating `ERMNeuralNetClassifier`.

We will be using SGD to train the classifier, using the same implementation from the previous homework. We have included the SGD implementation in `utils`.

In this task, you will complete the `predict` and `train_grad` methods. The first is for logging purposes, and the second is for calculating the gradients w.r.t. trainable parameters. Like `train_obj`, `train_grad` takes in and sets the parameters of the model and does a forward pass to calculate the loss on data `X, y`  indexed by the batch indices. `train_grad` finally does a backward propagation and returns the gradients w.r.t. trainable parameters.

In [11]:
class ERMNeuralNetClassifier(linclass.Classifier):
    '''
    Neural network trained by minimizing the empirical risk with SGD,
    w.r.t. some loss function.
    '''
    def __init__(self, model: Module, loss: Loss, **kwargs):
        '''
        Args:
            model: A neural network object with initialized parameters.
            loss: A loss function.
        '''
        super().__init__(**kwargs)
        self.model = model
        self.params0 = self.model.get_params()
        self.params = None
        self.loss = loss

    def predict(self, X: np.ndarray) -> np.ndarray:
        '''
        Returns predicted labels for data.

        Args:
            X: Data features. shape (m, d_in) where d_in is the number of
                input features of self.model.

        Returns:
            shape (m)
        '''
        assert self.params is not None, "Classifier hasn't been fit!"

        # Switch to evaluation mode to disable dropout
        self.model.eval_mode()

        #### TASK 5 CODE
        self.model.forward(X)
        ypred = np.argmax(self.model._output, axis = 1)
        return ypred
        #### TASK 5 CODE

    def fit(self, X: np.ndarray, y: np.ndarray, **sgd_kwargs):
        '''
        Fits the classifier on dataset.

        Args:
            X: Data features. shape (m, d_in) where d_in is the number of input
                features of self.model.
            y: Data labels, 0 <= y_i < k. shape (m)
        '''
        assert X.shape[0] == y.shape[0]

        m = X.shape[0]

        # Define training objective
        def train_obj(params: np.ndarray, batch: Optional[np.ndarray] = None) -> float:
            '''
            Calculates the training objective with parameters on a batch of training samples.

            Args:
                params: Trainable parameters, in the same format as self.model.get_params().
                batch: (default None) Indices of samples to calculate objective on. If None,
                    calculate objective on all samples.
            '''
            if batch is None:
                # All data is in a batch
                batch = slice(None)

            self.model.set_params(params)

            # Forward pass
            self.model.forward(X[batch])
            self.loss.forward(self.model._output, y[batch])

            loss_val = self.loss._output
            return loss_val

        # Define training gradient
        def train_grad(params: np.ndarray, batch: Optional[np.ndarray] = None) -> np.ndarray:
            '''
            Returns the gradient of the training objective w.r.t. parameters,
            calculated on a batch of training samples.

            Args:
                params: Trainable parameters, in the same format as self.model.get_params().
                batch: (default None) Indices of samples to calculate objective on. If None,
                    calculate objective on all samples.
            '''
            if batch is None:
                # All data is in a batch
                batch = slice(None)

            self.model.set_params(params)
            #### TASK 5 CODE
            # Forward pass
            self.model.forward(X[batch])
            # Backward pass
            self.loss.forward(self.model._output, y[batch])
            self.loss.backward()
            self.model.backward(self.loss._grad_input)
            
            grad_params = self.model.get_grad_params()
            #### TASK 5 CODE
            return grad_params

        self.sgd_loggers = [
            utils.SGDLogger('train_obj', train_obj, can_display=True, per_epoch=True),
        ] + sgd_kwargs.pop('loggers', [])

        # Switch to training mode to enable dropout, if present in the model
        self.model.train_mode()

        # Optimize using SGD
        self.params = utils.SGD(
            self.params0,
            train_grad,
            m,
            loggers=self.sgd_loggers,
            **sgd_kwargs
        )

### Training the Neural Network Classifier

We will now train a classifier using your implementation. We will use a toy example first. You should check that your training objective decreases with training, and that you get better than random training error (think what the error of random prediction is when the number of labels is 5).

After this, we will train a classifier and evaluate it on a real-world dataset.

In [12]:
n_train = 50
k = 5
X_train = np.random.rand(n_train, 10)
y_train = np.random.choice(k, n_train)

model = Sequential([
    Linear(10, 20),
    ReLU(),
    Linear(20, k)
])
loss = MultiLogisticLoss(k=k)
clf = ERMNeuralNetClassifier(model, loss)

clf.fit(X_train, y_train, eta=0.01, n_epochs=1000,
        verbose=True, verbose_epoch_interval=100)

y_train_pred = clf.predict(X_train)
train_err = utils.empirical_err(y_train, y_train_pred)

print(f'train_err: {train_err:5f}')


--- Epoch: 0, train_obj: 1.773089387369215
--- Epoch: 100, train_obj: 1.6264845126203757
--- Epoch: 200, train_obj: 1.5823367309118779
--- Epoch: 300, train_obj: 1.5478415524391145
--- Epoch: 400, train_obj: 1.5176034728379904
--- Epoch: 500, train_obj: 1.4935995581072277
--- Epoch: 600, train_obj: 1.472814521869054
--- Epoch: 700, train_obj: 1.4544168260579875
--- Epoch: 800, train_obj: 1.43755934963062
--- Epoch: 900, train_obj: 1.4217601665228434
train_err: 0.600000


### [Task 6] Fashion MNIST dataset

As mentioned, for this assigment, we will use the [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) dataset. It is a slightly harder 10-class classification problem than the MNIST dataset you worked with last week but is set up in the same way (10 classes, 28x28 images).

We provide you with `.npy` files of the data to use. The code below loads the training data and labels. Your task is to submit labels for the test set.

We provide an example network for inspiration. You should build a better model, perhaps using deeper models or wider layers (just examples).

#### **Endgame**

Finally, upload the predicted test labels to [this kaggle competition](https://www.kaggle.com/t/e7ec8dd1f8e64aa2997d82edd5e8f6eb), where we will first evaluate it on a slice of the test set to show your standings. Kaggle allows resubmissions, but beware of submitting relentlessly to climb up in the public leaderboard---you'd be overfitting to the test set. After the deadline, we will test it on secret data, which the public leaderboard does not show you, to see how well your predictor generalizes. We recommend using validation or cross-validation for developing your model and then submitting to Kaggle.

**Your grade will not depend on your performance relative to others in the class**. We simply want you to try your best on getting good performance. Any concepts you learnt in this class or code you developed in this class are fair game. You are also welcome to use methods not directly covered in this class, but **the work must be your own** -- i.e., you must implement and train the models yourself, even if the architecture is inspired by something you saw elsewhere.

**Report your Kaggle username here so we can know where you are on the leaderboard!**
#### **Your User Name: letranconghung, under name Hung L.**

In [13]:
train_data = np.load('fmnist_train.npy', allow_pickle=True).item()
test_data = np.load('fmnist_test.npy', allow_pickle=True).item()

X = train_data['data']
y = train_data['labels']
X_test = test_data['data']

# Preprocessing X
X = X.reshape((X.shape[0], -1))
if X.max() > 1: X = X / 255.

X_test = X_test.reshape((X_test.shape[0], -1))
if X_test.max() > 1: X_test = X_test / 255.

# Split into Xfm_train, yfm_train, Xfm_val, yfm_val
Xfm_train, yfm_train, Xfm_val, yfm_val = utils.create_split(X, y, 0.8)

models = [
    Sequential([
        Linear(Xfm_train.shape[1], 400),
        ReLU(),
        Dropout(0.1),
        Linear(400, 10)
    ]),
    Sequential([
        Linear(Xfm_train.shape[1], 2000),
        ReLU(),
        Dropout(0.1),
        Linear(2000, 10)
    ]),
    Sequential([
        Linear(Xfm_train.shape[1], 1000),
        ReLU(),
        Dropout(0.1),
        Linear(1000, 800),
        ReLU(),
        Dropout(0.1),
        Linear(800, 10)
    ]),
    Sequential([
        Linear(Xfm_train.shape[1], 1000),
        ReLU(),
        Dropout(0.1),
        Linear(1000, 800),
        ReLU(),
        Dropout(0.1),
        Linear(800, 400),
        ReLU(),
        Dropout(0.1),
        Linear(400, 10)
    ])
]
best_clf = None
best_val_err = np.inf
for model in models:
    loss = MultiLogisticLoss(k=10)
    # An example of a sequential network
    clf = ERMNeuralNetClassifier(model, loss)

    sgd_kwargs = {
        'batch_size': 128,
        'n_epochs': 600,
        'eta': 0.01,
        'verbose': True, # Enable printing INSIDE SGD
        'verbose_epoch_interval': 1,
    }

    clf.fit(Xfm_train, yfm_train, **sgd_kwargs)

    yfm_train_pred = clf.predict(Xfm_train)
    train_err = utils.empirical_err(yfm_train, yfm_train_pred)

    yfm_val_pred = clf.predict(Xfm_val)
    val_err = utils.empirical_err(yfm_val, yfm_val_pred)

    print(f'train_err: {train_err:5f}, val_err: {val_err:5f}')
    if val_err < best_val_err:
        best_clf = clf

#### TASK 6 CODE
y_test_preds = best_clf.predict(X_test)
#### TASK 6 CODE


--- Epoch: 0, train_obj: 0.7854294944265104
--- Epoch: 1, train_obj: 0.6623864912903151
--- Epoch: 2, train_obj: 0.6071661195308878
--- Epoch: 3, train_obj: 0.5694127644985907
--- Epoch: 4, train_obj: 0.5463399283574302
--- Epoch: 5, train_obj: 0.5285976057118981
--- Epoch: 6, train_obj: 0.5086578028712795
--- Epoch: 7, train_obj: 0.5006866748628566
--- Epoch: 8, train_obj: 0.4852357124796578
--- Epoch: 9, train_obj: 0.4758477859411868
--- Epoch: 10, train_obj: 0.46814120539534143
--- Epoch: 11, train_obj: 0.46585802449710073
--- Epoch: 12, train_obj: 0.4558283914960319
--- Epoch: 13, train_obj: 0.4475244773133152
--- Epoch: 14, train_obj: 0.44510178859304583
--- Epoch: 15, train_obj: 0.4385863373353356
--- Epoch: 16, train_obj: 0.4325293898891585
--- Epoch: 17, train_obj: 0.42858014797898236
--- Epoch: 18, train_obj: 0.427492642358077
--- Epoch: 19, train_obj: 0.42553744881359384
--- Epoch: 20, train_obj: 0.415288612835887
--- Epoch: 21, train_obj: 0.41071249582683944
--- Epoch: 22, 

#### Package output for uploading to Kaggle

In order to upload your results to Kaggle, load the test data, use your model to predict outputs, and then write the outputs to a csv file. You will upload the csv file stored at `fname` to the Kaggle competition.

In [16]:
print(best_clf.model)

In [17]:
# Save the CSV file of labels. UPLOAD THIS FILE
fname = 'fmnist_test_pred.csv'
output = np.vstack((np.arange(y_test_preds.shape[0]), y_test_preds)).T
np.savetxt(fname, output, fmt="%d", delimiter=',', comments='', header='id,label')

## [OPTIONAL] Extension: Convolutional Networks

In practice, fully-connected neural networks are hard to scale to images -- the number of units required grows significantly. However, in images there is a lot of local structure that we can exploit, in particular by focusing on local features and sharing weights for features derived from different parts of the image. Another way to think about this is as passing a convolutional filter over the whole image. This is similar to what we did in the Viola-Jones face detector, except that in this case, we will learn the parameter values (and therefore the filter itself) rather than pre-setting them. We did not cover convolutional neural networks in a lot of detail this quarter, but you can look at [this course](https://cs231n.github.io/) and in particular [this set of notes](https://cs231n.github.io/convolutional-networks/) for a good introduction to the topic.

To implement Neural Networks, we could take the approach that we did in the previous part of the homework, and build them ourselves from ground up. However, the implementation above is not very optimized, and we cannot run it on special hardware. To address this, there are several available libraries for building, training, and even deploying deep learning models. Of these, [Pytorch](https://pytorch.org/) is a very commonly used one. We'll give an example of how to load data from a pre-loaded dataset, how to set up a convolutional neural network, and how to train it. Feel free to experiment with this.

In [ ]:
!pip install torch torchvision

In [ ]:
# Import pytorch modules
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Import pytorch vision modules (preprocessing transforms, datasets)
from torchvision import transforms
from torchvision.datasets import FashionMNIST

In [ ]:
torch.manual_seed(SEED)

#### Load Data, set up `DataLoader`

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])
fmnist = FashionMNIST("/content/", train=True, transform=transform, download=True)
train_size = int(0.8*fmnist.targets.shape[0])
train_dataset, val_dataset = torch.utils.data.random_split(fmnist, [train_size, fmnist.targets.shape[0]-train_size])

In [ ]:
train_data = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
validation_data = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=True)

#### Implement ConvNet

In [ ]:
class ConvNet(nn.Module):
    def __init__(self, n_filters=6, start_fc_units=100, activation=F.relu):
        super(ConvNet, self).__init__()
        pass

    def forward(self, X):
        pass


#### Loops for Training and Evaluating

In [ ]:
# adapted from https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html
def train_loop(
    dataloader,
    model,
    loss_fn,
    optimizer,
    epochs=10,
    log_epoch_interval: int = 1
):
    size = len(dataloader.dataset)
    for ep in range(epochs):
        loss_val = 0.
        for batch, (X, y) in enumerate(dataloader):
            # Compute prediction and loss

            # Backpropagation

            continue


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

#### Loss and Optimizer

In [ ]:
cn1 = ConvNet(n_filters=2)
optimizer = torch.optim.SGD(cn1.parameters(), lr=0.1)
loss_fn = nn.CrossEntropyLoss()

#### Train

In [ ]:
train_loop(train_data, cn1, loss_fn, optimizer, epochs=10)

#### Test

In [ ]:
test_loop(validation_data, cn1, loss_fn)